# BoligPortal Scraper (with Selenium)

### Relevant Libraries

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from bs4 import BeautifulSoup as bs

from datetime import datetime
import os

import pandas as pd
import numpy as np

### ProgressBar Function to follow the scraper's progress

In [2]:
from functions import printProgressBar

### Selenium start-up

In [3]:
# <<< Insert your starting URL in the line below >>>

# First page of ads in all desired areas of Copenhagen
start_url='https://www.boligportal.dk/find?placeIds=19%2C817%2C14%2C49%2C106%2C24%2C44&housingTypes=1%2C3%2C4&maxRent=10500' 

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(start_url)
sleep(1)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/home/jan/.wdm/drivers/chromedriver/linux64/84.0.4147.30/chromedriver] found in cache


### Creating a results folder

In [5]:
# Create folder for the .csv results

wd = "/home/jan/Code/Scraper_Boligportal/results"

now = datetime.now()
current_time = now.strftime("%d%m%y")
dirName = current_time

try:
    # Create target Directory
    os.mkdir(os.path.join(wd, dirName))
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  240820  already exists


### URLs to scrape

In [6]:
# Extract the number of pages from the last Pagination button at the bottom of the page

soup = bs(driver.page_source, 'html')
num_pages = soup.find_all('a', {'class': 'PaginationControls__page'})[-1].getText() 
num_pages = int(num_pages)

# Extract number of ads
num_ads = num_pages * 18

In [7]:
# Create a range of numbers to generate the URLs to scrape
start_record = np.arange(0, num_ads, 18)

In [8]:
# Generate URLs to scrape, corresponding to page numbers (1, 2, 3, etc..)

urls = []

for i in start_record:
    url = start_url + '&startRecord={}'.format(i)
    urls.append(url)

### Scraping

In [9]:
# Selenium will go through each page (from the URLs list) and extract the AdCardWrappers
# Then, it will extract each AdCard's information

data = []

for url in urls:
    
    driver.get(url)
    sleep(1)
    soup = bs(driver.page_source, 'html')
    
    ads_buttons = soup.find_all('div', {'class': 'AdCardWrapper'})
    
    for ad in ads_buttons:
    
        try:
            title = ad.find('div', {'class': 'AdCard__title'}).getText() 
        except:
            title = -1

        try:
            price = ad.find('div', {'class': 'AdCard__price'}).getText() 
        except:
            price = -1

        try:
            location = ad.find('div', {'class': 'AdCard__location'}).getText() 
        except:
            location = -1

        try:
            date = ad.find('div', {'class': 'AdCard__date'}).getText() 
        except:
            date = -1

        try:
            link = 'boligportal.dk' + ad.find('a', {'itemprop' : 'url'}).get('href')
        except:
            link = -1

        try:
            description = ad.find('div', {'class': 'AdCard__description'}).getText() 
        except:
            description = -1


        data.append({"Title": title,
                     "Price": price,
                     "Location": location,
                     "When": date,
                     "Description": description,
                     "URL": link})

    printProgressBar(urls.index(url), len(urls), prefix = 'Scraping...')

In [10]:
# Scraped data preview
db pd.DataFrame(data)

,Title,Price,Location,When,Description,URL
0,1 værelses lejlighed på 32 m²,"10.100,-","København K, Store Kongensgade",Fremhævet,Her er en unik mulighed for at flytte ind i de...,boligportal.dk/lejebolig/dp/1-vaerelses-lejlig...
1,3 værelses lejlighed på 70 m²,"10.271,-","København S, Else Alfelts Vej",Fremhævet,Nyopført ejendom med 2 og 3 værelses lejlighed...,boligportal.dk/lejebolig/dp/3-vaerelses-lejlig...
2,1 værelses lejlighed på 45 m²,"8.500,-","København NV, Gravervænget",Fremhævet,Rummelig et værelses lejlighed 15 minutter fra...,boligportal.dk/lejebolig/dp/1-vaerelses-lejlig...
3,1 værelses lejlighed på 33 m²,"9.950,-","København K, Store Kongensgade",3 timer siden,1- OG 2-VÆRELSES BOLIGER DEN GAMLE POLITISTATI...,boligportal.dk/lejebolig/dp/1-vaerelses-lejlig...
4,2 værelses lejlighed på 47 m²,"3.500,-","København NV, Vilhelm Birkedals Vej",5 timer siden,En super hyggelig og møbleret egoist-lejlighed...,boligportal.dk/lejebolig/dp/2-vaerelses-lejlig...
...,...,...,...,...,...,...
199,2 værelses lejlighed på 45 m²,"9.245,-","København Ø, Willemoesgade",9 måneder siden,God lejlighed i Willemoesgade-kvarteret \nDenn...,boligportal.dk/lejebolig/dp/2-vaerelses-lejlig...
200,2 værelses lejlighed på 40 m²,"9.800,-","København Ø, Landskronagade",et år siden,Rigtig god lejlighed tæt på offentlig transpor...,boligportal.dk/lejebolig/dp/2-vaerelses-lejlig...
201,2 værelses lejlighed på 40 m²,"9.800,-","København Ø, Landskronagade",et år siden,Rigtig god lejlighed tæt på offentlig transpor...,boligportal.dk/lejebolig/dp/2-vaerelses-lejlig...
202,1 værelses lejlighed på 27 m²,"6.199,-","København NV, Dortheavej",2 år siden,Få lejlighed med eget køkken og bad i et af de...,boligportal.dk/lejebolig/dp/1-vaerelses-lejlig...


In [11]:
# Create a dataframe with the data

df = pd.DataFrame(data)

In [12]:
#df = df.drop_duplicates(subset=None, keep='first', inplace=False)

### Functions to clean/manipulate the dataframe

In [13]:
def get_rooms(row):
    
    ''' Function to use with df.apply. Creates a Rooms column by extracting the number of rooms from the Ad title '''
    
    if 'Værelse' in row.Title:
        return 0
    
    else:
        try:
            if len([int(s) for s in row.Title.split() if s.isdigit()]) == 1:
                return 1
            else:
                return [int(s) for s in row.Title.split() if s.isdigit()][0]
        except:
            return -1

In [14]:
def get_m2(row):
    
    ''' Function to use with df.apply. Creates a m2 column by extracting the squared meters from the Ad title '''
    
    try:
        
        try:
            return [int(s) for s in row.Title.split() if s.isdigit()][1]
        except:
            return [int(s) for s in row.Title.split() if s.isdigit()][0]
    
    except: 
        return -1

In [15]:
def en_et_to_one(column):
    
    '''To use on the column containing the age of the Ad. Replaces 'en and 'et' with 1, Fremhævet with 0, removes 'siden' '''
    
    column = column.str.replace('Fremhævet', '0')
    column = column.str.replace('siden', '')
    column = column.str.replace('en','1', 1)
    column = column.str.replace('et','1', 1)
    
    return column

In [16]:
def posted_x_days_ago(row):
    
    ''' Extracts age of post in terms of days '''

    if 'dag' in row.When:
        return [int(s) for s in row.When.split() if s.isdigit()][0]
    
    elif 'måned' in row.When:
        return [int(s) for s in row.When.split() if s.isdigit()][0] * 30
    
    elif 'år' in row.When:
        return [int(s) for s in row.When.split() if s.isdigit()][0] * 365

    else:
        return 0

### Dataframe cleaning/manipulation

In [17]:
df['Rooms'] = df.apply(lambda row: get_rooms(row), axis = 1) 

In [18]:
df['m2'] = df.apply(lambda row: get_m2(row), axis = 1)

In [19]:
df['Price'] = df['Price'].apply(lambda row: row.replace(",-", ""))

In [20]:
df[['Neighbourhood','Street']] = df.Location.str.split(", ", expand=True)
#df = df.drop('Empty', axis = 1)

In [21]:
df.When = en_et_to_one(df.When)
df['Posted days ago'] = df.apply(lambda row: posted_x_days_ago(row), axis = 1)
df = df.drop('When', axis = 1)

In [22]:
df = df[['Title', 'Price', 'Rooms', 'm2', 'Neighbourhood', 'Street', 'Posted days ago', 'Description', 'URL']]

In [23]:
df = df.sort_values(by = 'Posted days ago', ascending = True)

In [30]:
df[df["Posted days ago"]==0]

,Title,Price,Rooms,m2,Neighbourhood,Street,Posted days ago,Description,URL
0,1 værelses lejlighed på 32 m²,10.100,1,32,København K,Store Kongensgade,0,Her er en unik mulighed for at flytte ind i de...,boligportal.dk/lejebolig/dp/1-vaerelses-lejlig...
1,3 værelses lejlighed på 70 m²,10.271,3,70,København S,Else Alfelts Vej,0,Nyopført ejendom med 2 og 3 værelses lejlighed...,boligportal.dk/lejebolig/dp/3-vaerelses-lejlig...
2,1 værelses lejlighed på 45 m²,8.500,1,45,København NV,Gravervænget,0,Rummelig et værelses lejlighed 15 minutter fra...,boligportal.dk/lejebolig/dp/1-vaerelses-lejlig...
3,1 værelses lejlighed på 33 m²,9.950,1,33,København K,Store Kongensgade,0,1- OG 2-VÆRELSES BOLIGER DEN GAMLE POLITISTATI...,boligportal.dk/lejebolig/dp/1-vaerelses-lejlig...
4,2 værelses lejlighed på 47 m²,3.500,2,47,København NV,Vilhelm Birkedals Vej,0,En super hyggelig og møbleret egoist-lejlighed...,boligportal.dk/lejebolig/dp/2-vaerelses-lejlig...
5,2 værelses lejlighed på 41 m²,10.500,2,41,København K,Gernersgade,0,Nyrenoveret lejlighed i Nyboder 5 min fra Øste...,boligportal.dk/lejebolig/dp/2-vaerelses-lejlig...
6,2 værelses lejlighed på 55 m²,7.000,2,55,København N,Fafnersgade,0,Pendler bolig udlejes - ikke tilgængelig i wee...,boligportal.dk/lejebolig/dp/2-vaerelses-lejlig...
7,1 værelses lejlighed på 20 m²,5.900,1,20,København K,Larslejsstræde,0,Fin og ekstremt godt udnyttet 1-værelses lejli...,boligportal.dk/lejebolig/dp/1-vaerelses-lejlig...
8,1 værelses lejlighed på 44 m²,9.500,1,44,København K,Schacksgade,0,Rummelig og renoveret et værelses lejlighed i ...,boligportal.dk/lejebolig/dp/1-vaerelses-lejlig...
9,2 værelses lejlighed på 58 m²,10.500,2,58,København S,Grækenlandsvej,0,Nice furnsihed Apartment. \r\n\r\nAll kitchen ...,boligportal.dk/lejebolig/dp/2-vaerelses-lejlig...


### Export

In [24]:
df.to_csv(os.path.join(wd, dirName,'BoligPortal_KPH_{}.csv'.format(datetime.now().strftime("%d%m%y"))), index = True, header = True)